# Azure Ransomware Detection Lab

## Step 1: Import Python Packages

In [1]:
!pip install azure-identity azure-mgmt-loganalytics azure-monitor-query azure-mgmt-subscription &>/dev/null

from azure.identity import DeviceCodeCredential                   # Needed to authenticate with Azure
from azure.mgmt.loganalytics import LogAnalyticsManagementClient  # Allows interaction with Log Analytics
from azure.mgmt.subscription import SubscriptionClient            # Allows us to get the subscription ID
from azure.monitor.query import LogsQueryClient                   # Used to issue queries to Log Analytics
from datetime import timedelta, datetime                          # Time conversions
import pandas as pd                                               # Used to manipulate data

## Step 2: Authenticate with Azure

In [2]:
credential = DeviceCodeCredential()
credential.authenticate()
subscriptions_client = SubscriptionClient(credential)
query_client = LogsQueryClient(credential)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BDXYPT7GL to authenticate.


## Step 3: Get Data from Log Analytics Workspace

In [3]:
# Get first subscription
subscriptions = subscriptions_client.subscriptions.list()
for subscription in subscriptions:
    subscription_id = subscription.subscription_id
    break

# Get workspace_id
log_analytics_client = LogAnalyticsManagementClient(credential, subscription_id)
workspaces = log_analytics_client.workspaces.list()
for workspace in workspaces:
    if workspace.name == 'sherlocklaw':
        workspace_id = workspace.customer_id

# Get all StorageBlobLog data within the last day
query = """
StorageBlobLogs
"""
response = query_client.query_workspace(workspace_id, query, timespan=timedelta(days=1))
data = response.tables
for table in data:
    df = pd.DataFrame(data=table.rows, columns=table.columns)
pd.set_option('display.max_colwidth', None)
df

,TenantId,TimeGenerated,AccountName,Location,Protocol,OperationName,AuthenticationType,StatusCode,StatusText,DurationMs,...,SasExpiryStatus,MetricResponseType,SourceUri,DestinationUri,AccessTier,SourceAccessTier,RehydratePriority,SourceSystem,Type,_ResourceId
0,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 13:29:16.189148+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetBlobServiceProperties,TrustedAccess,200,Success,3,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
1,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 14:45:00.105102+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetBlobServiceProperties,TrustedAccess,200,Success,4,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
2,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 14:45:01.613983+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetBlobServiceProperties,AccountKey,200,Success,4,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
3,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 14:45:01.783098+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetContainerProperties,AccountKey,200,Success,4,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
4,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 14:45:01.791054+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetContainerProperties,AccountKey,200,Success,4,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 21:59:53.849415+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,PutBlob,AccountKey,201,Success,5,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
625,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 01:18:08.866052+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetBlobServiceProperties,TrustedAccess,200,Success,3,...,,Success,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
626,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 05:04:12.751049+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetContainerProperties,TrustedAccess,404,ContainerNotFound,3,...,,ClientOtherError,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default
627,7e8bff31-486e-4ba3-9c8c-257b9b5f759f,2025-03-14 05:04:12.758326+00:00,sherlockeqahz7pefp8y,eastus,HTTPS,GetContainerProperties,TrustedAccess,404,ContainerNotFound,2,...,,ClientOtherError,,,,Invalid,,Azure,StorageBlobLogs,/subscriptions/5bea71d0-410e-4adb-969d-a9038cd8016f/resourcegroups/sherlockrg/providers/microsoft.storage/storageaccounts/sherlockeqahz7pefp8y/blobservices/default


## Step 4: Detection Ransomware

In [4]:
def analyze_storagebloblogs_data(df):
    """Function to discover ransomware"""
    
    # Store GetBlob events into a separate DataFrame
    get_objects = df[df['OperationName'] == 'GetBlob']
    get_objects = get_objects.copy()
    get_objects['TimeGenerated'] = pd.to_datetime(get_objects['TimeGenerated'])

    
    # Store PutBlob events into a separate DataFrame
    put_objects = df[df['OperationName'] == 'PutBlob']
    put_objects = put_objects.copy()
    put_objects['TimeGenerated'] = pd.to_datetime(put_objects['TimeGenerated'])
    
    # Store DeleteBlob events into a separate DataFrame
    delete_objects = df[df['OperationName'] == 'DeleteBlob']
    delete_objects = delete_objects.copy()
    delete_objects['TimeGenerated'] = pd.to_datetime(delete_objects['TimeGenerated'])
    
    # Create empty results list
    results = []
    
    # Iterate through every GetBlob record
    for _, get_row in get_objects.iterrows():
        """
        Iterate through each row in get_objects and extract relevant details
          (ObjectKey, AccountName, TimeGenerated)
        """
        file_key = get_row['ObjectKey']
        account_name = get_row['AccountName']
        event_time = get_row['TimeGenerated']
        
        """
        Filters put_objects to find operations by the same account where the
          object name starts with but does not end with the file_key and
          occurred after the extracted event_time
        """
        put_candidates = put_objects[
            (put_objects['AccountName'] == account_name) &
            (~put_objects['ObjectKey'].str.endswith(file_key)) &
            (put_objects['ObjectKey'].str.startswith(file_key)) &
            (put_objects['TimeGenerated'] > event_time)
        ]
        
        """
        Filters the delete_objects DataFrame to find operations by the same
          account where the object key matches file_key and the operation
          occurred after the extracted event_time
        """
        delete_candidates = delete_objects[
            (delete_objects['AccountName'] == account_name) &
            (delete_objects['ObjectKey'] == file_key) &
            (delete_objects['TimeGenerated'] > event_time)
        ]
        """
        Checks if there are both upload (put_candidates) and delete
          (delete_candidates) operations, and if so, appends details about
          the original file, the account involved, the download time, the
          uploaded files, and a deletion confirmation to the results list
        """
        if not put_candidates.empty and not delete_candidates.empty:
            results.append({
                'OriginalFile': file_key,
                'AccountName': account_name,
                'DownloadedAt': event_time,
                'UploadedFiles': list(put_candidates['ObjectKey']),
                'Deleted': True
            })
    return pd.DataFrame(results)

analyze_storagebloblogs_data(df)

,OriginalFile,AccountName,DownloadedAt,UploadedFiles,Deleted
0,/sherlockeqahz7pefp8y/private/Abe_Jouaneton.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.124477+00:00,[/sherlockeqahz7pefp8y/private/Abe_Jouaneton.txt.enc],True
1,/sherlockeqahz7pefp8y/private/Aeriel_Clowser.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.224199+00:00,[/sherlockeqahz7pefp8y/private/Aeriel_Clowser.txt.enc],True
2,/sherlockeqahz7pefp8y/private/Arlyne_Boddie.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.327617+00:00,[/sherlockeqahz7pefp8y/private/Arlyne_Boddie.txt.enc],True
3,/sherlockeqahz7pefp8y/private/Armando_Perrygo.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.414840+00:00,[/sherlockeqahz7pefp8y/private/Armando_Perrygo.txt.enc],True
4,/sherlockeqahz7pefp8y/private/Britta_Ybarra.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.519286+00:00,[/sherlockeqahz7pefp8y/private/Britta_Ybarra.txt.enc],True
5,/sherlockeqahz7pefp8y/private/Carole_Marzelo.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.625628+00:00,[/sherlockeqahz7pefp8y/private/Carole_Marzelo.txt.enc],True
6,/sherlockeqahz7pefp8y/private/Carter_Eakeley.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.708676+00:00,[/sherlockeqahz7pefp8y/private/Carter_Eakeley.txt.enc],True
7,/sherlockeqahz7pefp8y/private/Catharine_Wiggett.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.810843+00:00,[/sherlockeqahz7pefp8y/private/Catharine_Wiggett.txt.enc],True
8,/sherlockeqahz7pefp8y/private/Cherlyn_Goodyer.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:50.915914+00:00,[/sherlockeqahz7pefp8y/private/Cherlyn_Goodyer.txt.enc],True
9,/sherlockeqahz7pefp8y/private/Darda_Woollends.txt,sherlockeqahz7pefp8y,2025-03-14 21:59:51.000370+00:00,[/sherlockeqahz7pefp8y/private/Darda_Woollends.txt.enc],True
